In [1]:

# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "unsupervised_learning"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [2]:
np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1

angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X = np.empty((m, 3))
X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)

In [3]:
# 特異値分解

X_centerd = X - X.mean(axis=0)
U, s, Vt = np.linalg.svd(X_centerd)
c1 = Vt.T[:, 0]
c2 = Vt.T[:, 1]
print("Vt：",Vt)
print("c1：",c1)
print("c2：",c2)

Vt： [[ 0.93636116  0.29854881  0.18465208]
 [-0.34027485  0.90119108  0.2684542 ]
 [-0.08626012 -0.31420255  0.94542898]]
c1： [0.93636116 0.29854881 0.18465208]
c2： [-0.34027485  0.90119108  0.2684542 ]


In [4]:
# 低次のd次元への射影

W2 = Vt.T[:, :2]
print("W2：", W2)
X2d = X_centerd.dot(W2)
print("X2d：", X2d)

W2： [[ 0.93636116 -0.34027485]
 [ 0.29854881  0.90119108]
 [ 0.18465208  0.2684542 ]]
X2d： [[-1.26203346 -0.42067648]
 [ 0.08001485  0.35272239]
 [-1.17545763 -0.36085729]
 [-0.89305601  0.30862856]
 [-0.73016287  0.25404049]
 [ 1.10436914 -0.20204953]
 [-1.27265808 -0.46781247]
 [ 0.44933007 -0.67736663]
 [ 1.09356195  0.04467792]
 [ 0.66177325  0.28651264]
 [-1.04466138  0.11244353]
 [ 1.05932502 -0.31189109]
 [-1.13761426 -0.14576655]
 [-1.16044117 -0.36481599]
 [ 1.00167625 -0.39422008]
 [-0.2750406   0.34391089]
 [ 0.45624787 -0.69707573]
 [ 0.79706574  0.26870969]
 [ 0.66924929 -0.65520024]
 [-1.30679728 -0.37671343]
 [ 0.6626586   0.32706423]
 [-1.25387588 -0.56043928]
 [-1.04046987  0.08727672]
 [-1.26047729 -0.1571074 ]
 [ 1.09786649 -0.38643428]
 [ 0.7130973  -0.64941523]
 [-0.17786909  0.43609071]
 [ 1.02975735 -0.33747452]
 [-0.94552283  0.22833268]
 [ 0.80994916  0.33810729]
 [ 0.20189175  0.3514758 ]
 [-1.34219411 -0.42415687]
 [ 0.13599883  0.37258632]
 [ 0.8206931  -0.5

In [5]:
# sklearnでPCA

from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X2D = pca.fit_transform(X)

In [6]:
X2D

array([[ 1.26203346,  0.42067648],
       [-0.08001485, -0.35272239],
       [ 1.17545763,  0.36085729],
       [ 0.89305601, -0.30862856],
       [ 0.73016287, -0.25404049],
       [-1.10436914,  0.20204953],
       [ 1.27265808,  0.46781247],
       [-0.44933007,  0.67736663],
       [-1.09356195, -0.04467792],
       [-0.66177325, -0.28651264],
       [ 1.04466138, -0.11244353],
       [-1.05932502,  0.31189109],
       [ 1.13761426,  0.14576655],
       [ 1.16044117,  0.36481599],
       [-1.00167625,  0.39422008],
       [ 0.2750406 , -0.34391089],
       [-0.45624787,  0.69707573],
       [-0.79706574, -0.26870969],
       [-0.66924929,  0.65520024],
       [ 1.30679728,  0.37671343],
       [-0.6626586 , -0.32706423],
       [ 1.25387588,  0.56043928],
       [ 1.04046987, -0.08727672],
       [ 1.26047729,  0.1571074 ],
       [-1.09786649,  0.38643428],
       [-0.7130973 ,  0.64941523],
       [ 0.17786909, -0.43609071],
       [-1.02975735,  0.33747452],
       [ 0.94552283,

In [7]:
# 因子寄与率（分散の全体に対する割合）

pca.explained_variance_ratio_

array([0.84248607, 0.14631839])

In [8]:
# MNISTの準備

from six.moves import urllib
try:
    from sklearn.datasets import fetch_openml
    mnist = fetch_openml('mnist_784', version=1)
    mnist.target = mnist.target.astype(np.int64)
except ImportError:
    from sklearn.datasets import fetch_mldata
    mnist = fetch_mldata('MNIST original')
from sklearn.model_selection import train_test_split

X = mnist["data"]
y = mnist["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [9]:
# 次元を削減する前にPCAを計算

pca = PCA()
pca.fit(X_train)
# 要素の値を合計
cumsum = np.cumsum(pca.explained_variance_ratio_)
# 因子寄与率の合計が95%以上になるようにする
d = np.argmax(cumsum >= 0.95) + 1
d

154

In [10]:
# 以上の実装を簡潔にする

pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X_train)

In [11]:
# 次数
pca.n_components_

154

In [12]:
# データセットの再構築

pca = PCA(n_components = 154)
X_reduced = pca.fit_transform(X_train)
X_recoverd = pca.inverse_transform(X_reduced)

In [13]:
# 追加学習型PCA(ミニバッチに分割)

from sklearn.decomposition import IncrementalPCA

n_batches = 100
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_train, n_batches):
    inc_pca.partial_fit(X_batch)
X_reduced = inc_pca.transform(X_train)

In [14]:
# memmapを使用した追加学習PCA(必要な配列のみ取り出す)

# filename = "my_mnist.data"
# m, n = X_train.shape
# X_mm = np.memmap(filename, dtype="float32", mode="readonly"
#                 ,shape=(m,n))

# データからミニバッチ内のデータ数を切り捨て除算し
# ミニバッチ数を計算

# batch_size = m // n_batches
# inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
# inc_pca.fit(X_mm)

In [15]:
# del X_mm

In [16]:
# ランダム化PCA
rnd_pca = PCA(n_components=154, svd_solver="randomized")
X_reduced = rnd_pca.fit_transform(X_train)

In [17]:
# データの準備(スイスロール)

from sklearn.datasets import make_swiss_roll

X, t = make_swiss_roll(n_samples=1000, noise=0.2, random_state=41)
y = t > 6.9

In [18]:
# カーネルPCA
from sklearn.decomposition import KernelPCA

rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.04)
X_reduced = rbf_pca.fit_transform(X)

In [19]:
# モデルの選択とパラメータチューニング


from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ("kpca", KernelPCA(n_components=2)),
    ("log_reg", LogisticRegression())
])

param_grid = [{
    "kpca__gamma":np.linspace(0.03, 0.05, 10),
    "kpca__kernel": ["rbf","sigmoid"]
}]

grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X,y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('kpca', KernelPCA(alpha=1.0, coef0=1, copy_X=True, degree=3, eigen_solver='auto',
     fit_inverse_transform=False, gamma=None, kernel='linear',
     kernel_params=None, max_iter=None, n_components=2, n_jobs=None,
     random_state=None, remove_zero_eig=False, tol=0)), ('log_reg', LogisticRe...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'kpca__gamma': array([0.03   , 0.03222, 0.03444, 0.03667, 0.03889, 0.04111, 0.04333,
       0.04556, 0.04778, 0.05   ]), 'kpca__kernel': ['rbf', 'sigmoid']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [20]:
print(grid_search.best_params_)

{'kpca__gamma': 0.04555555555555556, 'kpca__kernel': 'rbf'}


In [21]:
rbf_pca = KernelPCA(n_components = 2, gamma=0.456, kernel="rbf",
                   fit_inverse_transform=True)
X_reduced = rbf_pca.fit_transform(X)
X_preimage = rbf_pca.inverse_transform(X_reduced) #再構築

In [22]:
from sklearn.metrics import mean_squared_error

# 再構築誤差
mean_squared_error(X, X_preimage)

40.94193423447866

In [23]:
# LLE 

from sklearn.manifold import LocallyLinearEmbedding

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)
X_reduced = lle.fit_transform(X)

In [24]:
#演習9

#データの準備

X_train = mnist['data'][:60000]
y_train = mnist['target'][:60000]

X_test = mnist['data'][60000:]
y_test = mnist['target'][60000:]

In [25]:
from sklearn.ensemble import RandomForestClassifier

before_rnd_clf = RandomForestClassifier(random_state=42)
%timeit before_rnd_clf.fit(X_train, y_train) # training speed

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


7.17 s ± 48.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
from sklearn.metrics import accuracy_score

y_before_pred = before_rnd_clf.predict(X_test)
accuracy_score(y_test, y_before_pred)

0.9492

In [27]:
# 次元削減
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X_train)

In [28]:
#　新しいランダムフォレスト分類器

after_rnd_clf = RandomForestClassifier(random_state=42)
%timeit after_rnd_clf.fit(X_reduced, y_train) # training speed

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


19.5 s ± 638 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
X_test_reduced = pca.transform(X_test)
y_after_pred = after_rnd_clf.predict(X_test_reduced)
accuracy_score(y_test,y_after_pred)

0.9009